In [1]:
import numpy as np
import torch
import torch.utils.data as data
import torch.nn as nn
import torch.nn.functional as nnf
import torch.optim as optim
import os

os.chdir('..')

from collections import defaultdict
from optim.optimizer import CosineWarmupScheduler
from run.train import training
from run.test import test
from tqdm import tqdm

import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from mpl_toolkits.axes_grid1 import make_axes_locatable
from matplotlib.cm import coolwarm
plt.set_cmap('cividis')


from scipy import spatial
from sklearn.preprocessing import Normalizer
import itertools
from itertools import tee

# Ensure that all operations are deterministic on GPU (if used) for reproducibility
torch.backends.cudnn.determinstic = True
torch.backends.cudnn.benchmark = False

device = torch.device("cuda:0") if torch.cuda.is_available() else torch.device("cpu")
print("Device:", device)

Device: cuda:0


<Figure size 432x288 with 0 Axes>

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F


class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(9 , 100)  
        self.fc2 = nn.Linear(100, 100)
        self.d = nn.Dropout(p=0.5)
        self.fc4 = nn.Linear(100, 6)
        

    def forward(self, x):
        #Applying relu non-linearity
        x = self.fc1(x)
        x = F.leaky_relu(x, negative_slope =0.1)
        x = self.fc2(x)
        x = F.leaky_relu(x, negative_slope =0.1) 
        x = self.d(x)
        x = self.fc4(x)
        return x


model = Net()


In [3]:
def normalize(inp_data, out_data):
    #t_start
    inp_data[:,0] /= 600    # 10 mins
    #t_operating
    #inp_data[:,1] /= (86400*5)    # 1 day
    inp_data[:,1] = np.log(inp_data[:,1]+1)/20
    #t_tracktime
    #inp_data[:,2] /= 2592000   # 1 month
    inp_data[:,2] = np.log(inp_data[:,2]+1)/20
    #t_cumsum
    #inp_data[:,2] /= 2592000   # 1 month
    inp_data[:,3] = np.log(inp_data[:,3]+1)/30
    # power 
    inp_data[:,4] /= 500
    #guidevane
    inp_data[:,5] /= 180
    # draft_pressure
    inp_data[:,6] /= 500
    # spiral_pressure
    inp_data[:,7] /= 10000
    # rotational speed
    inp_data[:,8] /= 500
    
    ## output
    out_data /= 2000
    
    return inp_data, out_data
    
    
    

In [4]:
f_dataset = np.load('Input_to_NN_notNAN.npz', allow_pickle = True)
print(f_dataset['X'].shape, f_dataset['Y'].shape )

#inp_data = f_dataset['X'][:, [0,1,2,4,6,7,8,9,10] ]
inp_data = f_dataset['X'][:, [0,1,2,3,4,6,7,8,9] ]
out_data = f_dataset['Y'] 

print(np.mean(inp_data, axis =0))
print(np.mean(out_data, axis =0))

print(np.count_nonzero(np.isnan(f_dataset['X'])), np.count_nonzero(np.isnan(f_dataset['Y'])))

#inp_data[:,4:8] = Normalizer().fit_transform(f_dataset['X'][:,4:8])
#out_data = Normalizer().fit_transform(f_dataset['Y'])

inp_data, out_data = normalize(inp_data, out_data)

print(np.mean(inp_data, axis =0))
print(np.mean(out_data, axis =0))

print('Max :', np.max(inp_data, axis =0))
print('Max :',np.max(out_data, axis =0))

print(inp_data[:,8])

print(np.max(inp_data[:,0]))


(1350000, 10) (1350000, 6)
[1.49223927e+02 1.16750037e+05 8.51013559e+05 9.64145371e+10
 2.98626508e+02 1.55861493e+02 5.31218745e+03 1.07874637e+02
 9.97639259e-01]
[1608.72025976 1487.72319511 1688.32415064 1602.36688526 1636.97079258
 1678.00840345]
0 0
[2.48706546e-01 5.49544591e-01 6.62822448e-01 8.27220053e-01
 5.97253016e-01 8.65897185e-01 1.06243749e+01 1.07874637e-02
 1.99527852e-03]
[0.80436013 0.7438616  0.84416208 0.80118344 0.8184854  0.8390042 ]
Max : [4.08333333e-01 6.44532536e-01 7.21967553e-01 8.59446855e-01
 6.46604664e-01 1.49086662e+00 1.10255430e+01 1.08368752e-02
 2.00000000e-03]
Max : [0.81010815 0.74734008 0.84716078 0.80274691 0.81949943 0.84172924]
[0.002 0.002 0.002 ... 0.002 0.002 0.002]
0.4083333333333333


In [5]:
class SeqDataset(data.Dataset):

    def __init__(self, dataset , labelset):
        super().__init__()
        self.data = torch.from_numpy(dataset)
        self.labels = torch.from_numpy(labelset)
        
               
    def __len__(self):
        return self.data.size()[0]

    def __getitem__(self, idx):
        inp_data = self.data[idx]
        out_data = self.labels[idx]
        return inp_data, out_data

In [6]:
full_dataset = SeqDataset(np.array(inp_data , dtype= 'float32'), np.array(out_data, dtype = 'float32') )
print(len(full_dataset))

1350000


In [7]:
train_size = int(0.7 * len(full_dataset))
val_size = int(0.1 * len(full_dataset.data))
test_size = len(full_dataset.data)-train_size -val_size
batch = 256
print(train_size, val_size, test_size)
train_dataset, val_dataset, test_dataset = torch.utils.data.random_split(full_dataset, [train_size,val_size, test_size])

train_loader = data.DataLoader(train_dataset, batch_size=batch, shuffle=True, drop_last=True, pin_memory=True)
val_loader   = data.DataLoader(val_dataset, batch_size=batch)
test_loader  = data.DataLoader(test_dataset, batch_size=batch)

944999 135000 270001


In [8]:
model = model.to(device)
    
print(model)

## Initializing optimizer
lr=1e-3
optimizer = torch.optim.Adam(model.parameters(), lr=lr)

Net(
  (fc1): Linear(in_features=9, out_features=100, bias=True)
  (fc2): Linear(in_features=100, out_features=100, bias=True)
  (d): Dropout(p=0.5, inplace=False)
  (fc4): Linear(in_features=100, out_features=6, bias=True)
)


In [9]:
def calculate_loss(data, labels, model, device=torch.device("cuda:0")):
    target = model.forward(data)
    loss = nnf.l1_loss(target, labels)
    l1_lambda = 0.01
    l1_reg = torch.tensor(0.).to(device)
    for name, params in model.named_parameters() :
        if 'fc2' in name:
            l1_reg += torch.abs(params).sum()
    loss += l1_lambda * l1_reg
    return loss, target                  # target = predictedc_bolt_tensile


def training(model, train_loader, val_loader, epochs, optimizer, checkpoint_path, filename,
             device=torch.device("cuda:0"), aux_loss=False):
    min_val_loss = np.inf
    for epoch in tqdm(range(epochs)):
        train_loss = 0.0
        model.train()
        for i, (data, labels) in enumerate(train_loader):
            optimizer.zero_grad()
            if torch.cuda.is_available():
                data, labels = data.to(device), labels.to(device)
            # Calculate Train Loss
            loss, _ = calculate_loss(data, labels, model)
            # back-propagate Train loss
            loss.backward()
            optimizer.step()
            train_loss += loss.item()

        val_loss = 0.0
        model.eval()
        for data, labels in val_loader:
            if torch.cuda.is_available():
                data, labels = data.to(device), labels.to(device)
            # Calculate Validation Loss
            loss, _ = calculate_loss(data, labels, model)
            val_loss += loss.item()

        print(
            f'Epoch {epoch + 1} \t\t Training Loss: {train_loss / len(train_loader)} \t\t'
            f' Validation Loss: {val_loss / len(val_loader)}')

        if min_val_loss > val_loss:
            print('trigger times: 0')
            trigger_times = 0
            print(f'Validation Loss Decreased({min_val_loss:.6f}--->{val_loss:.6f}) \t Saving The Model')
            min_val_loss = val_loss
            # Saving State Dict
            os.makedirs(checkpoint_path, exist_ok=True)
            pretrained_filename = os.path.join(checkpoint_path, filename)
            torch.save({
                'epoch': epoch + 1,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'val_loss': val_loss}, pretrained_filename)

    return model

In [ ]:



available_filename = ['simple_nn_model.pth']
# Path to the folder where the pretrained models are saved
CHECKPOINT_PATH = "models/trained_models/"


# Check whether pretrained model exists. If yes, load it and skip training
pretrained_filename = os.path.join(CHECKPOINT_PATH, available_filename[0]  )
print(pretrained_filename)
if os.path.isfile(pretrained_filename):
    print("Found pretrained model, loading...")
    checkpoint = torch.load(pretrained_filename)
    model.load_state_dict(checkpoint['model_state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    epoch = checkpoint['epoch']
    val_loss = checkpoint['val_loss']
    print(epoch)
else:
    model = training(model, train_loader, val_loader, 100, optimizer, CHECKPOINT_PATH, available_filename[0],device)

models/trained_models/simple_nn_model.pth


  1%|▊                                                                                 | 1/100 [00:23<38:29, 23.33s/it]

Epoch 1 		 Training Loss: 0.1017941765346398 		 Validation Loss: 0.012372908140109344
trigger times: 0
Validation Loss Decreased(inf--->6.532895) 	 Saving The Model


  2%|█▋                                                                                | 2/100 [00:45<36:55, 22.60s/it]

Epoch 2 		 Training Loss: 0.01239572674721334 		 Validation Loss: 0.012387494759450696


...

Epoch 18 		 Training Loss: 0.012136308098349653 		 Validation Loss: 0.012446105330265269


 19%|███████████████▍                                                                 | 19/100 [07:17<32:28, 24.06s/it]

Epoch 19 		 Training Loss: 0.012122644321446463 		 Validation Loss: 0.012088634632880601


 20%|████████████████▏                                                                | 20/100 [07:40<31:40, 23.76s/it]

Epoch 20 		 Training Loss: 0.01211024922263498 		 Validation Loss: 0.011987733458888462
